In [1]:
#set WD
import os
os.getcwd()
os.chdir('C:\\Users\\AT003502\\Documents\Phishing project')

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import sklearn 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import re
%matplotlib notebook
from urllib.parse import urlparse
import whois
import datetime
import time
#import httplib2
import urllib
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import tldextract
#import tldextract
import pickle

In [34]:
DATA_PATH = 'hmrc.csv'
dataframe = pd.read_csv(DATA_PATH, index_col=0,encoding="ISO-8859-1")
#dataframe.head

In [ ]:
def df_fun(url):
    index=(0,)
    columns=['url']
    df=pd.DataFrame(url,index=index,columns=columns)
    return (df)
#dot="."
#process_urls(dataframe)
#dataframe=df_fun("http://sesfl.g./asdlc.com")
#dataframe.head()

In [4]:
def process_urls(url):
    dot="."
    import tldextract
    for row_idx, row in dataframe.iterrows():
        dataframe.loc[row_idx, 'length'] = (len(row['url']))
        dataframe.loc[row_idx, 'RU'] = 1.0 if '.ru' in row['url'] else 0.0
        dataframe.loc[row_idx, 'Com'] = 1.0 if '.com' in row['url'] else 0.0
        dataframe.loc[row_idx, 'Org'] = 1.0 if '.org' in row['url'] else 0.0
        dataframe.loc[row_idx, 'Co'] = 1.0 if '.co.' in row['url'] else 0.0
        dataframe.loc[row_idx, 'IP_add'] = 1.0 if re.search( r'[0-9]+(?:\.[0-9]+){3}',(str(row['url']))) else 0.0
        dataframe.loc[row_idx,'has_at_symbol'] = 1.0 if '@' in (str(row['url'])) else 0.0
        dataframe.loc[row_idx,'pref_suf']  = 1.0 if '-' in row['url'] else 0.0
        dataframe.loc[row_idx,'no_dots'] = (str(row['url'])).count(".")
        dataframe.loc[row_idx, 'redirection'] = (str(row['url'])).count("//")
        dataframe.loc[row_idx, 'subdirectory'] =(str(row['url'])).count("/")
        dataframe.loc[row_idx,'domain'] =tldextract.extract(str(row['url'])).domain
        dataframe.loc[row_idx,'sub_domain'] =tldextract.extract(str(row['url'])).subdomain
        dataframe.loc[row_idx,'suffix'] =tldextract.extract((str(row['url']))).suffix
        dataframe.loc[row_idx,'dom_subdom'] =((str(row['domain']))+dot+(str(row['sub_domain'])))
        dataframe.loc[row_idx,'dots_per_dom'] = 1 if (str(row['dom_subdom'])).count(".") > 1 else 0
        dataframe.loc[row_idx,'dashes'] = (str(row['url'])).count("-") 
        dataframe.loc[row_idx, 'sub_dom_www'] = 1.0 if re.search( r'www',(str(row['sub_domain']))) else 0.0
        dataframe.loc[row_idx, 'dom_sub_www'] = 1.0 if re.search( r'www',(str(row['dom_subdom']))) else 0.0
            



In [7]:
process_urls(dataframe)


In [8]:
dataframe.isnull().sum()


url              0
type             0
length           0
RU               0
Com              0
Org              0
Co               0
IP_add           0
has_at_symbol    0
pref_suf         0
no_dots          0
redirection      0
subdirectory     0
domain           0
sub_domain       0
suffix           0
dom_subdom       0
dots_per_dom     0
dashes           0
sub_dom_www      0
dom_sub_www      0
dtype: int64

In [9]:
dataframe=dataframe.drop(["domain", "sub_domain", "suffix","dom_subdom","url"], axis=1)
#dataframe.head()

In [10]:
tag=pd.get_dummies(dataframe["type"],drop_first=True)
tag.tail(5)
dataframe=pd.concat([dataframe, tag], axis=1)
#dataframe.head()

In [11]:
dataframe=dataframe.sample(frac=1)

In [12]:
X=dataframe.drop(["type","Malicious"], axis=1)
y= dataframe["Malicious"]
X.head(5)

,length,RU,Com,Org,Co,IP_add,has_at_symbol,pref_suf,no_dots,redirection,subdirectory,dots_per_dom,dashes,sub_dom_www,dom_sub_www
Index,,,,,,,,,,,,,,,
3648,119.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,9.0,0.0,2.0,0.0,0.0
2714,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,5.0,0.0,0.0,0.0,0.0
3150,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0
3751,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,6.0,0.0,0.0,0.0,0.0
2389,89.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.cross_validation import train_test_split

C:\Users\AT003502\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
logmodel=LogisticRegression()

In [17]:
logmodel.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
predictions = logmodel.predict(X_test)


In [19]:
from sklearn.metrics import classification_report

In [20]:
classification_report(y_test,predictions)

'             precision    recall  f1-score   support\n\n          0       0.82      0.78      0.80       185\n          1       0.94      0.95      0.94       615\n\navg / total       0.91      0.91      0.91       800\n'

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
accuracy_score(y_test,predictions)

0.91

## Improving the model

In [23]:
X=dataframe.drop(["type","Malicious"], axis=1)
X=dataframe.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]].values
y=dataframe.iloc[:,16].values
#dataframe.info()

In [24]:
X

array([[119.,   0.,   0., ...,   2.,   0.,   0.],
       [ 68.,   0.,   0., ...,   0.,   0.,   0.],
       [ 32.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [116.,   0.,   0., ...,   0.,   0.,   0.],
       [117.,   0.,   0., ...,   0.,   0.,   0.],
       [ 72.,   0.,   0., ...,   3.,   0.,   0.]])

In [25]:
y

array([0, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [29]:
model=LogisticRegression(random_state=0)
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
y_pred=model.predict(X_test)

In [31]:
accuracy_score(y_test,predictions)

0.91

### Pickle the model

In [32]:
filename='model_1.sav'
pkl_filename = "pickle_model.pkl"

In [33]:
import pickle
import pickle
f = open('model.pkl', 'wb')
pickle.dump(model, f)
f.close()


PermissionError: [Errno 13] Permission denied: 'model.pkl'